Julia for Python (and C++) programmers
========================================

Why Julia?
----------

**Julia** is a relatively young (2012) programming language designed to be effective for scientific workflows - the developers specifically call out Fortran and MATLAB as predecessors in this area. 
Compared to these, Julia has much of the dynamic and interactive expressiveness of languages such as *Python* (including functionality that is only provided there by the third-party *NumPy* library), whilst leveraging just-in-time compilation and specialisation to allow performance approaching (and sometimes better than) high-performance compiled languages such as *C*, *Modern Fortran* and *Rust*.

### List of Julia properties

+ High-performance (within 0.5 order of magnitude of C; often at parity)
+ Fully Unicode supporting - including LaTeX markup support in REPL, and natural mathematical symbols for common operations
+ First-class support in Jupyter (it's the *Ju* bit of the name)
+ Supports object-oriented programming (with *multiple dispatch*), Functional-style method chaining, and other modern paradigms
+ Dynamic typing, with efficient function specialisation (via JIT) and optional typing supported deeply in language (unlike Python)



Example Julia Code
-------------------

In [1]:
function myfunc(x, y)
    if x > y
        "X is greater than Y\n"
    else
        "Y is greater than X\n"
    end
end

print(myfunc(2,3))

print(myfunc(2.0, 1))

Y is greater than X
X is greater than Y


As you can see, Julia's syntax is somewhere between that of Python and MATLAB in style, but it's easy to follow.
One subtle thing we're using in this example is that *all* expressions return a value - the "value" of the *if* chain is the string that is created as the only operation in that chain... and thus the "value" of the function, without an explicit *return* statement, is simply the value of the last expression that was executed before the function ended.

We could write this more explicitly as:

In [2]:
function verbosefunc(x, y)
    if x > y 
        return "X is greater than Y\n"
    else 
        return "Y is greater than X\n"
    end
end

verbosefunc (generic function with 1 method)

and of course *explicit* return statements are necessary in some places to control flow.

As we mentioned, Julia has a JIT which applies to all code written in it - the first time a unit of code is executed, the JIT will compile direct to host machine code. 
We can see this better with a longer function that requires actual effort (thanks to @Moelf), using the @time macro to get the execution time:

In [3]:
function go_faster(a)
    trace = 0.0
    for i in axes(a, 1)
        trace += tanh(a[i, i])
    end
    return a .+ trace
end

go_faster (generic function with 1 method)

In [4]:
x = reshape(0:99, 10, 10)

@time go_faster(x)

  0.072604 seconds (269.89 k allocations: 18.373 MiB, 99.96% compilation time)


10×10 Matrix{Float64}:
  9.0  19.0  29.0  39.0  49.0  59.0  69.0  79.0  89.0   99.0
 10.0  20.0  30.0  40.0  50.0  60.0  70.0  80.0  90.0  100.0
 11.0  21.0  31.0  41.0  51.0  61.0  71.0  81.0  91.0  101.0
 12.0  22.0  32.0  42.0  52.0  62.0  72.0  82.0  92.0  102.0
 13.0  23.0  33.0  43.0  53.0  63.0  73.0  83.0  93.0  103.0
 14.0  24.0  34.0  44.0  54.0  64.0  74.0  84.0  94.0  104.0
 15.0  25.0  35.0  45.0  55.0  65.0  75.0  85.0  95.0  105.0
 16.0  26.0  36.0  46.0  56.0  66.0  76.0  86.0  96.0  106.0
 17.0  27.0  37.0  47.0  57.0  67.0  77.0  87.0  97.0  107.0
 18.0  28.0  38.0  48.0  58.0  68.0  78.0  88.0  98.0  108.0

As we can see, over 99.9% of the time taken was by the JIT. 
However, as Julia caches the result, subsequent executions are much faster, using the previously compiled code:


In [4]:
y = reshape(1:100, 10, 10)

@time go_faster(y)

  0.072999 seconds (269.89 k allocations: 18.373 MiB, 99.96% compilation time)


10×10 Matrix{Float64}:
 10.7616  20.7616  30.7616  40.7616  …  70.7616  80.7616  90.7616  100.762
 11.7616  21.7616  31.7616  41.7616     71.7616  81.7616  91.7616  101.762
 12.7616  22.7616  32.7616  42.7616     72.7616  82.7616  92.7616  102.762
 13.7616  23.7616  33.7616  43.7616     73.7616  83.7616  93.7616  103.762
 14.7616  24.7616  34.7616  44.7616     74.7616  84.7616  94.7616  104.762
 15.7616  25.7616  35.7616  45.7616  …  75.7616  85.7616  95.7616  105.762
 16.7616  26.7616  36.7616  46.7616     76.7616  86.7616  96.7616  106.762
 17.7616  27.7616  37.7616  47.7616     77.7616  87.7616  97.7616  107.762
 18.7616  28.7616  38.7616  48.7616     78.7616  88.7616  98.7616  108.762
 19.7616  29.7616  39.7616  49.7616     79.7616  89.7616  99.7616  109.762

Of course, it's possible to achieve similar effects using the various "JIT in Python" packages like *numba*, but those are additional add-ons to Python itself, and often require writing code in outwardly "unPythonic" ways.


A Quick Overview of Julia Syntax
----------------------------------

Much of Julia's syntax should be easy to pick up for users of Python or other modern languages. The example functions above demonstrate much of the control flow, but we'll go over the basics quickly, pointing out Juliaisms where we see them.

### Comments

Comments in Julia are made with a **#**


### Types 

Values in Julia can have various *Types*, depending on the kind of data they represent. In common with many 21st C languages, Julia expresses the relationship between Types as a *Type Hierarchy*, as seen below:

![The Julia Type Hierarchy, thanks to Uwe Hernandez Acosta](./images/numeric_types.png)

Only the leaves of the hierarchy are "concrete types" - types that a value can possess. The remaining types are "abstract types", existing to allow Julia to reason about common properties of two values of different types. 

A value of a particular concrete type also matches all of the abstract types above it in the hierarchy - a **Float64** value is also an **AbstractFloat**, a **Real** and a **Number**.

In Julia, the "<:" operator expresses the relation of "subtype" - so it is **true** that **Float64 <: AbstractFloat**

For the most part, as in Python, you can write a value and assume that Julia will pick the correct type for it automatically. 


### Variable Declaration and Types

Variables in Julia can be declared typelessly, like in Python - they must be declared with an assignment. In this case, they work like Python variables, taking on the type of whatever value is contained in them. Assigning a differently typed value to the variable changes its type.

Variables may use almost Unicode characters they like.

(from now on, we'll be using the **@show** macro, which outputs the current line, and its result, as part of the Cell output, in order to cut down on the number of cells we need)

In [5]:
myvar_α = 1 

@show typeof(myvar_α)

myvar_α = 2.0

@show typeof(myvar_α)

typeof(myvar_α) = Int64
typeof(myvar_α) = Float64


Float64

We can also choose to specify explicitly the type of a variable, by annotating it on first declaration with the special ::Type syntax.
This *binds* the specified type to the variable - meaning that values will be *converted* to the variable's type on assignment, if necessary.
(If Julia cannot convert a value without losing precision, it will instead throw an error)

This is a recurring approach in Julia - you *can* specify types, but if not, Julia will infer them if necessary from context.

In [6]:
myint::Int64 = 1 

@show typeof(myint)

myint = 2.0

@show typeof(myint)

myint  #note that this is an integer value 

typeof(myint) = Int64
typeof(myint) = Int64


2

In [7]:
myint = 2.5  #InexactError - Julia won't truncate a floating point value implicitly

LoadError: InexactError: Int64(2.5)

Julia also supports Complex values internally - the **im** keyword is used to represent *i*. Complex values support all the usual arithmetic operations transparently:

In [8]:
(2 + 3im) / (4im)

0.75 - 0.5im

### Composite Types ("Structs")

We can also define new composite types - where we have to provide some type information for the contained values. 
By default, composite types variables are *immutable*, like Python tuples with named elements.

In [9]:
struct my_vector
    x::Float64
    y::Float64
end

@show υϵκτωρ = my_vector(3.7, 6e7)

υϵκτωρ.x = 3.6

υϵκτωρ = my_vector(3.7, 6.0e7) = my_vector(3.7, 6.0e7)


LoadError: setfield!: immutable struct of type my_vector cannot be changed

We can also define explicitly *mutable* composite types, whose values can be modified, as well as generic composite types which expect a type parameter to be provided when instantiated.

In this case, "templated_vector" structs need to be given the type of their data - either explicitly, or implicitly (by inference from the values they're initialised with).

In [10]:
mutable struct mutable_vector
    x::Float64
    y::Float64
end

@show μυτ = mutable_vector(3.6, 7.8)

@show μυτ.x = 5

struct templated_vector{T<:Number}  #here, we can make any kind of templated_vector as long as the type is a Number
    x::T
    y::T
end

@show int_vec = templated_vector{Int64}(6,7)

@show float_vec = templated_vector(6.0,6.0)

μυτ = mutable_vector(3.6, 7.8) = mutable_vector(3.6, 7.8)
μυτ.x = 5 = 5
int_vec = templated_vector{Int64}(6, 7) = templated_vector{Int64}(6, 7)
float_vec = templated_vector(6.0, 6.0) = templated_vector{Float64}(6.0, 6.0)


templated_vector{Float64}(6.0, 6.0)

### Arrays, Dicts and other Composite Types

Julia supports multidimensional Arrays as first-class members of its type system, as you already may have noticed. 

Unlike C and C++, Arrays in Julia are not simply wrappers around pointer arithmetic - and unlike those languages and Python, multidimensional Arrays are not simply "arrays of single-dimensional arrays/lists". This allows Julia to provide advanced Array types - like SparseArray - that efficiently store data with particular symmetries or organisational properties.

Also, unlike all those languages, Julia Arrays are *column-major* - the left-most index is the one that varies most rapidly, as in Fortran and MatLab.

Arrays can be used like Python lists - to contain *any* mix of value types - but we usually don't want to do that because it's inefficient.

We can create "Array Literals" with a superficially familiar [] syntax, we can separate values by commas if we're making a 1-d array. We can also specify ranges with colons, and "list comprehension" type syntax.

For higher-dimensional arrays, we can nest brackets, using spaces, tabs or semicolons to concatenate the "columns" into a multidimensional structure... (there's a deep syntax for constructing high-dimensionality arrays which we won't cover in this intro!)

In [11]:
@show [1,2,3]

@show [1, 2.0, 'b']

@show [1:5]

@show [i^2 for i = 4:8]

@show [[1,2] [3,4]] #the Out[22] refers to this

[1, 2, 3] = [1, 2, 3]
[1, 2.0, 'b'] = Any[1, 2.0, 'b']
[1:5] = UnitRange{Int64}[1:5]
[i ^ 2 for i = 4:8] = [16, 25, 36, 49, 64]
[[1, 2] [3, 4]] = [1 3; 2 4]


2×2 Matrix{Int64}:
 1  3
 2  4

More usually, we will instead create Arrays via the constructor, or via the many utility functions that exist (similarly to NumPy).
In the first example we directly create an (uninitialised, hence 'random' element values) array of 64-bit integers... but that's not a very common requirement - notice that Arrays are effectively "templated types" like our struct above.

In [12]:
@show empty_array = Array{Int64}(undef,2)

α = @show zeros(Float64, 2, 3)

β = @show reshape(1:9, 3, 3)

empty_array = Array{Int64}(undef, 2) = [1675873130224, 1675873130256]
zeros(Float64, 2, 3) = [0.0 0.0 0.0; 0.0 0.0 0.0]
reshape(1:9, 3, 3) = [1 4 7; 2 5 8; 3 6 9]


3×3 reshape(::UnitRange{Int64}, 3, 3) with eltype Int64:
 1  4  7
 2  5  8
 3  6  9

Arrays are indexed via *all* of their indices in a single bracket, as they are entire objects even if multi-dimensional. 
As in Python, ":" allows us to specify a range to *slice* a particular index over - a bare ":" represents the entire range.

Julia arrays default to 1-indexing, not 0-indexing that you might be used to. The special keywords "begin" and "end" can also be used to refer to the first and last elements in an indexed range.

In [13]:
@show β[begin,end]

@show β[1,1:2]

@show β[:,2]

β[begin, end] = 7
β[1, 1:2] = [1, 4]
β[:, 2] = [4, 5, 6]


3-element Vector{Int64}:
 4
 5
 6

#### Tuples

Julia also supports immutable Arrays - Tuples - which, as in Python, are created using parenthesis ( ) , rather than [ ]. 
You can name the fields of a tuple if you want (using "=" to map names to values).

In [14]:
@show my_tuple = (2,3)

@show named_tuple = (a=1, b=7)

@show named_tuple.a

my_tuple = (2, 3) = (2, 3)
named_tuple = (a = 1, b = 7) = (a = 1, b = 7)
named_tuple.a = 1


1

Tuple expressions can also be used as part of Julia's support for "destructuring" values in composite types on assignment.
Here, we distribute the 4 values on the right into 4 variables on the left:

In [15]:
@show (a,b,c,d) = 9:13

@show b

(a, b, c, d) = 9:13 = 9:13
b = 10


10

### Dictionaries etc

Finally, we can make Dictionaries (for C++ programmers - HashMaps or AssociativeArrays) with the Dict constructor.
Unlike Python, the mapping between a key and a value is denoted with a "=>".
There's also a "dictionary comprehension" syntax for dictionaries that can be computed as an operation over ranges.

Dictionaries have a range of methods that work on them.

We can also make Sets and other advanced collections.

In [16]:
farmyard_sounds = Dict("🐮" => "moo", "sheep" => "baa", "pig" => "oink", "farmer" => "get off my land!")

@show farmyard_sounds["🐮"]

@show haskey(farmyard_sounds,"pig")

@show farmyard_sounds["🐱"] = "Meow"

@show Set(values(farmyard_sounds))

farmyard_sounds["🐮"] = "moo"
haskey(farmyard_sounds, "pig") = true
farmyard_sounds["🐱"] = "Meow" = "Meow"
Set(values(farmyard_sounds)) = Set(["get off my land!", "baa", "oink", "Meow", "moo"])


Set{String} with 5 elements:
  "get off my land!"
  "baa"
  "oink"
  "Meow"
  "moo"

### Strings

Strings in Julia are Unicode, encoded as UTF-8. They are delimited by double quotes, or triple-double-quotes (which allows you to use double quotes inside the string). Single quotes delimit *characters*, as in C, C++.

A string can be indexed into like an Array, but indexing is done byte-wise and Julia *will* throw an error if you try to index into the middle of a code-point. For iteration over the code-points in a string, Julia provides the *eachindex* method.

String concatenation is done with the * operator.

In [17]:
текст = "The Ukrainian for 'text' is 'текст' "   

@show текст[38] #trying to access position 37  would fail as it's in the middle of the 'с'

текст * """and the Hindi is "पाठ" """

текст[38] = 'т'


"The Ukrainian for 'text' is 'текст' and the Hindi is \"पाठ\" "

Strings also allow interpolation of expressions in Julia - using `$()` to contain the expression to be evaluated, or just `$expr` if `expr` is a single variable name.

In [18]:
@show "35 squared is $(35^2)"

"""The string текст is "$текст" """

"35 squared is $(35 ^ 2)" = "35 squared is 1225"


"The string текст is \"The Ukrainian for 'text' is 'текст' \" "

By contrast, single characters are delimited by single quotes, and are internally represented as 32-bit values (they default to UTF-32 representation).

In [19]:
alpha = 'α'

'α': Unicode U+03B1 (category Ll: Letter, lowercase)

## Control Flow

Julia delimits "blocks" of code using "begin" and "end" keywords - it's not dependant on white-space indenting like Python, although Julia *will* indent your code for you in a REPL or notebook, to make it more readable.

Explicit Loops look a lot like Python, with the addition of the explict "end" to close the block:

In [20]:
for i in 0:4
    println("$i×$i is $(i*i)")
end

i = 6
while i > 0 
    println("i = $i")
    i -= 1
end

0×0 is 0
1×1 is 1
2×2 is 4
3×3 is 9
4×4 is 16
i = 6
i = 5
i = 4
i = 3
i = 2
i = 1


Similarly, explicit branching looks quite familiar:

In [21]:
i = 5

if i > 3 
    println("$i > 3")
elseif i < 0
    println("$i is negative")
else
    println("$i is positive and ≤3")
end

5 > 3
